In [ ]:
import pandas as pd
import numpy as np

import os
# CHANGE THIS DIRECTORY
#os.chdir('/Users/una.lee/The Wonderful Company/Wonderful Education Data - DataTeam/Scripts/')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import winter 2021 NWEA scores
#nwea_winter = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\NWEAWinter.csv")
# import fall 2021 NWEA scores
nwea_fall = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Tests\AssessmentResults.csv")

#nwea_spring = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\Spring Projection Winter to Spring 2023.csv")

#dibs_boy = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\DIBELSBOY.csv")

#dibs_moy = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\MOYDIBELS.csv")

In [ ]:
# Use files generated from Fixing Grades for ILP Use in Secondary Final
# If scores needed are required use Scores Needed file that is generated for current grades
grades = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\SS\Grade Final Q3 Final HS.csv")
#fall_grades = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\Grades Semester One Secondary.csv")

# import student roster
#LIST STU ENR STU.SC STU.RS STU.ID STU.NM STU.FN STU.LN STU.LF STU.GP ENR.YR ENR.SC ENR.ED ENR.LD ENR.GR ENR.ED
roster = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Roster.xlsx")
roster = roster[(roster['Leave Date'].isnull()) & (roster['Year']==2024)].copy()
roster = roster.drop_duplicates()

# import list of SpED and teachers 
#LIST STU LN FN ID GR U1 U1? CSE.DI CAR CAR.PD CAR.TN TCH.TE CAR.DS CAR.DE IF CAR.PD = 0 from Patty **USE THIS ONE**
#LIST STU TCH IDN CSE STU.ID STU.NM STU.GR TCH.TE CSE.DI CSE.ED CSE.XD IF CSE.DI > 0 AND CSE.XD = NULL
sped = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\SPED.xlsx")
sped1 = sped.sort_values(['Student ID','Date Started']).copy()
sped2 = sped.drop_duplicates(subset=['Student ID'], keep='first')
sped2.loc[sped2[(sped2['Disability']>0) | (sped2['User1'].isin(['B','O','R','S']))].index, 'Student is SpED?'] = 'Student has an IEP'
sped2.loc[sped2[(sped2['User1']=='5')].index, 'Student is SpED?'] = 'Student has 504 Plan'

# import list of Teachers for students
#LIST STU ID LN FN GR SEC MST.RM MST TCH.TE CRS CRS.CO MST.PD
teachers = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\ES\TeacherStudents.xlsx")
#teachers_fall = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\Lost Hills ES\TeachersandPeriodQuarter2.xlsx")

#LIST STU GBU GBS GBA GBK GBR TCH STU.ID STU.NM STU.GR STU.SC GBK.GN GBR.CD GBK.NM GBK.PD TCH.TE GBU.D1 GBU.CSC GBU.CMK GBA.AN GBA.AD GBA.DD GBA.GC GBS.MK GBA.MX IF GBS.MK = " " AND GBS.MX > 0 AND GBA.GC = 1 AND GBA.DD >= GBR.SD AND GBA.DD <= GBR.ED AND GBK.SC = TCH.SC
missing_assignments = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\SS\MS\Missing Assignments.xlsx")

att = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Attendance.xlsx")

#LIST AHS STU STU.SC STU.ID STU.GR AHS.YR AHS.TD AHS.TRU IF AHS.YR = 2022-2023
truant = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Truant.xlsx")


In [ ]:
# import historical grades
#LIST STU HIS CRS STU.NM STU.GR STU.SC STU.ID HIS.SN HIS.CN HIS.MK HIS.GR HIS.TE HIS.YR HIS.CO HIS.CC his.rt CRS.CN CRS.CO CRS.E1
hg_grades = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\SS\A-G.csv")


In [ ]:
grades

In [ ]:
roster = roster.loc[roster['School'] == 8]
roster

In [ ]:
#start_date.to_csv('~/Documents/WCPA/Progress Reports/Output/District Start Date by ID 02222022.csv', index=False)


### SpED and teacher information

In [ ]:
#Add in Sped and teacher information
roster1 = roster.merge(sped2[['Student ID',
                             'Student is SpED?',
                             'Teacher name']],
                       on='Student ID', how='left').reset_index()
# Add in District Entry Date
roster2 = roster1
roster2.groupby('School')['Student ID'].count()


In [ ]:
roster2

In [ ]:
roster2 = roster2.drop(columns=['index', 'Res Schl', 'School1', 'Enter Date', 'Leave Date', 'Enter Date1'])
roster2

### NWEA

In [ ]:
def clean_nwea(nwea_df, growth_col, term):
    nwea_df1 = nwea_df[(nwea_df['GrowthMeasureYN']==True)&(nwea_fall['Course'].isin(['Reading','Math K-12']))].copy()

    nwea_df1.loc[nwea_df1[nwea_df1['TestPercentile'] <= 20].index,'NWEA Category'] = 'Significantly Below Grade Level'

    nwea_df1.loc[nwea_df1[(nwea_df1['TestPercentile'] > 20) &
             (nwea_df1['TestPercentile'] <= 40)].index,'NWEA Category'] = 'Below Grade Level'

    nwea_df1.loc[nwea_df1[(nwea_df1['TestPercentile'] > 40) &
             (nwea_df1['TestPercentile'] <= 60)].index,'NWEA Category'] = 'At Grade Level'

    nwea_df1.loc[nwea_df1[nwea_df1['TestPercentile'] > 60].index,'NWEA Category'] = 'Above Grade Level'

    nwea_df1.loc[nwea_df1[nwea_df1['TestPercentile'].isnull()].index,'NWEA Category'] = 'No Percentile Assigned'
    
    nwea_final = nwea_df1.pivot(index=['StudentID'], columns='Subject',values=['NWEA Category',growth_col]).reset_index()

    nwea_final.columns = ['StudentID',f"NWEA {term} Language Arts",
                      f"NWEA {term} Mathematics",
                      f"NWEA {term} Growth Goal Met - Language Arts",
                      f"NWEA {term} Growth Goal Met - Mathematics"]
    return nwea_final

#nwea_winter_clean = clean_nwea(nwea_winter,'FallToWinterMetProjectedGrowth','Winter 2021')
nwea_fall_clean = clean_nwea(nwea_fall,'FallToFallMetProjectedGrowth','Fall 2023')
#nwea_final = nwea_winter_clean.merge(nwea_fall_clean[['StudentID',
                                                      #'NWEA Fall 2021 Language Arts',
                                                      #'NWEA Fall 2021 Mathematics']], on='StudentID', how='outer')

In [ ]:
nwea_fall_clean

In [ ]:
nwea_fall_clean.info()

In [ ]:
nwea_fall_clean[:2]


In [ ]:
nwea_final = nwea_fall_clean

In [ ]:
roster2 = pd.merge(roster2, nwea_final, how='left', left_on='Student ID', right_on='StudentID')
roster2

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Delano HS Assessments", filename = "Delano HS Assessments"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(roster2)

### Attendance

In [ ]:
att

In [ ]:
att['Q1 Attendance'] = att['% Present'] *100
att = att.round({'Q1 Attendance': 0})
att

In [ ]:
final_att = att[['Student ID', 'Q1 Attendance', 'Unexcused Absences']]
final_att

### Missing Assignments

In [ ]:
ma  = missing_assignments

ma


In [ ]:
ma['Missing Assignments'] = 1
ma

In [ ]:
ma2 = ma.groupby(by=['Student ID'])['Missing Assignments'].sum().reset_index()
ma2

In [ ]:
roster2 = pd.merge(roster2, ma2, how='left', on='Student ID')
roster2

In [ ]:
roster2['Missing Assignments'] = roster2['Missing Assignments'].fillna(0)
roster2

## Tardies and Truancies

In [ ]:
truant

In [ ]:
truant = truant.groupby(by=['Student ID'])['Tardies', 'Days of Truancy'].sum().reset_index()
truant

In [ ]:
truant = truant[['Student ID', 'Tardies']]
truant

In [ ]:
att = pd.merge(final_att, truant, on='Student ID', how='outer')
att

In [ ]:
roster2 = pd.merge(roster2, att, how='left', on='Student ID')
roster2

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Delano HS Attendance", filename = "Delano HS Attendance"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(roster2)

### Current grades

In [ ]:
grades.info()


In [ ]:
grades

In [ ]:
#teachers = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2023-2024\Fall\Data\Teachers.xlsx")
#teachers

In [ ]:
#teachers = teachers[['Student ID', 'Teacher name', 'Course title']]
#teachers

In [ ]:
#grades = pd.merge(grades,teachers, how='outer', left_on = ['Student ID', 'Course title'], right_on=['Student ID', 'Course title'])
grades

In [ ]:
grades = grades.sort_values(['Student ID','Period'],ascending=True)
grades

In [ ]:
#grades = grades.drop_duplicates(subset=['Student ID', 'Course title'], keep='first')
#grades

In [ ]:
grades['Course Number'] = grades.groupby(['Student ID']).cumcount()+1

grades_courses = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Course title').reset_index()
grades_grades = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Mark1').reset_index()
grades_teachers = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Teacher name').reset_index()
grades_scores = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Score needed').reset_index()

grades_courses_name = {1:'Quarter 1 Course 1',
                 2:'Quarter 1 Course 2',
                 3:'Quarter 1 Course 3',
                 4:'Quarter 1 Course 4',
                 5:'Quarter 1 Course 5',
                 6:'Quarter 1 Course 6',
                 7:'Quarter 1 Course 7',
                 8:'Quarter 1 Course 8',
                 9:'Quarter 1 Course 9',
                     10:'Quarter 1 Course 10'}

grades_grades_name = {1:'Quarter 1 Course 1 Grade',
                 2:'Quarter 1 Course 2 Grade',
                 3:'Quarter 1 Course 3 Grade',
                 4:'Quarter 1 Course 4 Grade',
                 5:'Quarter 1 Course 5 Grade',
                 6:'Quarter 1 Course 6 Grade',
                 7:'Quarter 1 Course 7 Grade',
                 8:'Quarter 1 Course 8 Grade',
                 9:'Quarter 1 Course 9 Grade',
                    10:'Quarter 1 Course 10 Grade'}

grades_teachers_name = {1:'Quarter 1 Course 1 Teacher',
                 2:'Quarter 1 Course 2 Teacher',
                 3:'Quarter 1 Course 3 Teacher',
                 4:'Quarter 1 Course 4 Teacher',
                 5:'Quarter 1 Course 5 Teacher',
                 6:'Quarter 1 Course 6 Teacher',
                 7:'Quarter 1 Course 7 Teacher',
                 8:'Quarter 1 Course 8 Teacher',
                 9:'Quarter 1 Course 9 Teacher',
                      10:'Quarter 1 Course 10 Teacher'}

grades_scores_name = {1:'Quarter 1 Course 1 Score Needed',
                 2:'Quarter 1 Course 2 Score Needed',
                 3:'Quarter 1 Course 3 Score Needed',
                 4:'Quarter 1 Course 4 Score Needed',
                 5:'Quarter 1 Course 5 Score Needed',
                 6:'Quarter 1 Course 6 Score Needed',
                 7:'Quarter 1 Course 7 Score Needed',
                 8:'Quarter 1 Course 8 Score Needed',
                 9:'Quarter 1 Course 9 Score Needed',
                      10:'Quarter 1 Course 10 Score Needed'}

grades_courses.rename(columns=grades_courses_name, inplace=True)
grades_grades.rename(columns=grades_grades_name, inplace=True)
grades_teachers.rename(columns=grades_teachers_name, inplace=True)
grades_scores.rename(columns=grades_scores_name, inplace=True)

In [ ]:
grades_courses

In [ ]:
grades_grades

In [ ]:
grades_teachers

In [ ]:
grades_scores

In [ ]:
merge_one = pd.merge(grades_courses, grades_grades, how = 'outer', on = 'Student ID')
merge_one

In [ ]:
merge_two = pd.merge(merge_one, grades_teachers, how = 'outer', on = 'Student ID')
merge_two

In [ ]:
grades_final = pd.merge(merge_two, grades_scores, how = 'outer', on = 'Student ID')
grades_final

In [ ]:
roster2

In [ ]:
final = pd.merge(roster2, grades_final, how='left', on='Student ID')
final

In [ ]:
final.info()

In [ ]:
hg_grades

In [ ]:
final = pd.merge(final, hg_grades, how='left', on=['Student ID', 'School', 'Student Name', 'Grade'])
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Lost Hills HS Final Again", filename = "Lost Hills HS Final Again"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)